In [1]:
import random
import numpy as np
import torch
from exp.exp_long_term_forecasting import Exp_Long_Term_Forecast
from exp.exp_imputation import Exp_Imputation
from exp.exp_short_term_forecasting import Exp_Short_Term_Forecast
from exp.exp_anomaly_detection import Exp_Anomaly_Detection
from exp.exp_classification import Exp_Classification
from data_provider.data_creat import *
import akshare as ak
import datetime

In [25]:
class Args:
    '''基本配置'''
    # 选项：[long_term_forecast, short_term_forecast, imputation, classification, anomaly_detection]')
    task_name = 'long_term_forecast'
    is_training = 1
    model_id = 'Stock_96_96'
    # 模型名称，选项：[Autoformer, Transformer, TimesNet]
    model = 'TimesNet'
    
    '''股票数据获取'''
    fuquan = 'hfq'# 设置复权方式,adjust=空选择的不复权，qfq是前复权，应该用hfq后复权来进行量化分析
    period = 'daily' # 拉取时间周期{'daily', 'weekly', 'monthly'}
    start_date = '20161201'  # 下载数据的开始日期,0就是公司上市时间
    end_date = '-1'  # 下载数据的结束日期,如果0则到最后一天,如果-1是昨天.
    final_data_feat =  ['Open', 'Close', 'High', 'Low', 'K_valume','Amplitude', 'Change', 'Turnover', 'macd', 'rsi',\
        'kdj_k', 'kdj_d', 'twf_feat', 'lable']
    # 选择拼接前需要归一化的特征
    feat_nor_lise = ['Open', 'Close', 'High', 'Low', 'K_valume','Amplitude', 'Change', 'Turnover', 'macd', 'rsi', 'kdj_k', 'kdj_d', 'twf_feat',]
    label_n = 7 # 预测未来连续多少天的收益率
    zhangfu = 0.03  # 预测涨幅大于等于3%的为1，小于3%的为0
    label_ch = False  # 如果是True ，预测n天以后上涨大于变量zhangfu为1，小于为0
    
    
    '''数据加载'''
    # 数据集类型,选项：[ETTh1,ETTh2,ETTm1,ETTm2,custom,m4,PSM,MSL,SMAP,SMD,SWAT,UEA]
    data = 'custom'
    root_path = './dataset/Stock/'
    data_path = 'Stock.csv'
    # 预测任务 M:多变量预测多变量, S:单变量预测单变量, MS:多变量预测单变量
    features = 'MS'
    # 目标列名，S或MS任务中的目标特征
    target = 'OT'
    # 时间采集粒度，选项：[s:秒, t:分钟, h:小时, d:天, b:工作日, w:周, m:月]
    freq = 'd'
    # 模型检查点的位置
    checkpoints = './checkpoints/'

    '''预测任务'''
    # 输入序列长度
    seq_len = 96
    # 开始标记长度
    label_len = 48
    # 预测序列长度
    pred_len = 1
    # 季节模式（针对M4数据集）
    seasonal_patterns = 'Monthly'
    inverse = False    # 反转输出数据

    '''插补任务'''
    # 插补任务中数据丢失率
    mask_rate = 0.25

    '''异常检测任务'''
    # 异常检测中异常点占比
    anomaly_ratio = 0.25

    '''模型定义'''
    # TimesBlock 中傅里叶变换,频率排名前k个周期
    top_k = 5
    # Inception 中卷积核个数
    num_kernels = 6
    # encoder 输入特征数
    enc_in = 21
    # decoder 输入特征数
    dec_in = 21
    # 输出通道数
    c_out = 21
    # 线性层隐含神经元个数
    d_model = 32
    # FFN 层隐含神经元个数
    d_ff = 32
    # 多头注意力机制
    n_heads = 8
    # encoder 层数
    e_layers = 2
    # decoder 层数
    d_layers = 1
    # 滑动窗口长度
    moving_avg = 25
    # 对 Q 进行采样，对 Q 采样的因子数
    factor = 3
    # 是否下采样操作 pooling
    distil = True
    # dropout 率
    dropout = 0.1
    # 时间特征嵌入方式,选项：[timeF, fixed, learned]
    embed = 'timeF'
    # 激活函数类型
    activation = 'gelu'
    # 是否输出 attention
    output_attention = False

    '''优化'''
    # 并行核心数
    num_workers = 10
    # 实验轮数
    itr = 1
    # 训练迭代次数
    train_epochs = 10
    # batch size 大小
    batch_size = 32
    # early stopping 机制容忍次数
    patience = 3
    # 学习率
    learning_rate = 0.0001
    # 实验描述
    des = 'test'
    # 损失函数
    loss = 'MSE'
    # 学习率下降策略
    lradj = 'type1'
    # 使用混合精度训练
    use_amp = False

    '''GPU'''
    # 使用 gpu
    use_gpu = False
    gpu = 0
    # 使用多个 gpus
    use_multi_gpu = False
    # 多 gpu 的设备 id
    devices = '0,1,2,3'

    '''去平稳化投影仪参数'''
    # 投影仪的隐藏层维度（列表）
    p_hidden_dims = [128, 128]
    # 投影仪中的隐藏层数
    p_hidden_layers = 2


# 创建参数对象
args = Args()

In [3]:
from pyti.moving_average_convergence_divergence import moving_average_convergence_divergence as macd
from pyti.relative_strength_index import  relative_strength_index as rsi
from pyti.stochastic import percent_k as kdj_k
from pyti.stochastic import percent_d as kdj_d
from data_provider.func_util import *
from data_provider.func_stock import *
from data_provider.func_RankGauss import *

In [4]:
def add_data(raw_data,args):
    '''创建模型合适的数据集train_loader, valid_loader, test_loader'''

    # 提取列数据为NumPy数组
    open_data = raw_data['Open'].values
    close_data = raw_data['Close'].values
    high_data = raw_data['High'].values
    low_data = raw_data['Low'].values
    volume_data = raw_data['Volume'].values

    # 指标计算
    raw_data['macd'] = macd(close_data, 12, 26)
    raw_data['rsi'] = rsi(close_data, 14)
    raw_data['kdj_k'] = kdj_k(close_data, 9)
    raw_data['kdj_d'] = kdj_d(close_data, 9)
    raw_data['twf_feat'] = twf_feat(close_data, high_data, low_data, volume_data, 21)
    raw_data['K_volume'] = volume_data / 1000  # 注意列名修正为 'K_volume'
    print(f'添加数据以后形状： {raw_data.shape}')
    return raw_data

In [5]:
def sub_data(raw_data,args):
    '''
    添加预测目标lable列,并将所有的nan值填充为0'''
    lable_data = add_lable(raw_data,args.zhangfu,args.lable_n, args.lable_ch)
    '''
    删除第一列date列,删除前26行因为macd和cmf是空值,
    删除最后config['feat_n']行因为未来lable的n天的数据是空值,影响训练'''
    lable_data = lable_data.iloc[26:-args.lable_n, 1:]
    '''
    手动选择拼接前原始数据列'''
    final_data = lable_data[args.final_data_feat].copy() 
    print('拼接前data形状: ',final_data.shape)
    print('拼接前数据有: ',args.final_data_feat)
    '''
    如果标签为True,那么就对final_data的change,twf列进行归一化'''
    if args.feat_normalize == 1:
        for i in args.feat_nor_lise:
            final_data[i] = normalize(final_data[i])
        print('拼接前对数据进行归一化处理')
    elif args.feat_normalize == 2:
        for i in args.feat_nor_lise:
            final_data[i] = Standardization(final_data[i])
        print('拼接前对数据进行标准化处理')
    elif args.feat_normalize == 3:
        for i in args.feat_nor_lise:
            final_data[i] = QTF(final_data[i])
        print('拼接前对数据进行均匀分布转换QuantileTransformer处理')
    elif args.feat_normalize == 4:
        for i in args.feat_nor_lise:
            rg_data = final_data[i].values
            Rank = RankGauss_norm()
            final_data[i] = Rank.fit_transform(rg_data)
        print('拼接前对数据进行高斯标准化Rank Gaussian Normalization处理')
    elif args.feat_normalize == 5:
        for i in args.feat_nor_lise:
            final_data[i] = PowerTransformer(final_data[i])
        print('拼接前对数据进行高斯分布转换PowerTransformer处理')
    elif args.feat_normalize == 6:
        for i in args.feat_nor_lise:
            final_data[i] = RobustScaler(final_data[i])
        print('拼接前对数据进行鲁棒缩放RobustScaler处理')

In [23]:
def add_label(data, args):
    '''原始数据计算并添加预测标签'''
    data['Tom_Chg'] = (data['Close'].shift(-args.label_n) - data['Close'])/ data['Close']# 计算第n天到今天收益率
    if args.label_ch:
        # 初始化OT列为0
        data['OT'] = 0
        # 如果第n天到今天收益率data['Tom_Chg']大于0.5，那么label就等于1
        data.loc[data['Tom_Chg'] >= args.zhangfu, 'OT'] = 1
    else:
        data['OT'] = data['Tom_Chg']
    data = data.fillna(0)#将数据中的nan替换为0
    return data

In [26]:
stock_list = ['600028'] # 测试用
for i in stock_list:
    NUM = i
    # 获取数据
    raw_data = download_data(NUM, args)
    # 拼接数据
    a_data = add_data(raw_data.copy(), args)
    # 添加标签
    s_data = add_label(a_data.copy(), args)
    
    

获取数据时间为： 20161201 - 20231214
数据形状： (1712, 9)
添加数据以后形状： (1712, 15)


In [21]:
# 设置随机种子以确保结果可重现
fix_seed = 2021
random.seed(fix_seed)
torch.manual_seed(fix_seed)
np.random.seed(fix_seed)

In [25]:
# 检查并设置 GPU
args.use_gpu = True if torch.cuda.is_available() and args.use_gpu else False
if args.use_gpu and args.use_multi_gpu:
    args.devices = args.devices.replace(' ', '')
    device_ids = args.devices.split(',')
    args.device_ids = [int(id_) for id_ in device_ids]
    args.gpu = args.device_ids[0]

# 选择合适的实验类
if args.task_name == 'long_term_forecast':
    Exp = Exp_Long_Term_Forecast
elif args.task_name == 'short_term_forecast':
    Exp = Exp_Short_Term_Forecast
elif args.task_name == 'imputation':
    Exp = Exp_Imputation
elif args.task_name == 'anomaly_detection':
    Exp = Exp_Anomaly_Detection
elif args.task_name == 'classification':
    Exp = Exp_Classification
else:
    Exp = Exp_Long_Term_Forecast  # 默认情况

# 进行训练和测试
if args.is_training:
    for ii in range(args.itr):
        # 实验记录设置
        setting = '{}_{}_{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_fc{}_eb{}_dt{}_{}_{}'.format(
            args.task_name,
            args.model_id,
            args.model,
            args.data,
            args.features,
            args.seq_len,
            args.label_len,
            args.pred_len,
            args.d_model,
            args.n_heads,
            args.e_layers,
            args.d_layers,
            args.d_ff,
            args.factor,
            args.embed,
            args.distil,
            args.des, ii)

        exp = Exp(args)  # 设置实验
        print('>>>>>>>开始训练 : {}>>>>>>>>>>>>>>>>>>>>>>>>>>'.format(setting))
        exp.train(setting)

        print('>>>>>>>测试 : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
        exp.test(setting)
        if args.use_gpu:
            torch.cuda.empty_cache()
else:
    ii = 0
    setting = '{}_{}_{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_fc{}_eb{}_dt{}_{}_{}'.format(
        args.task_name,
        args.model_id,
        args.model,
        args.data,
        args.features,
        args.seq_len,
        args.label_len,
        args.pred_len,
        args.d_model,
        args.n_heads,
        args.e_layers,
        args.d_layers,
        args.d_ff,
        args.factor,
        args.embed,
        args.distil,
        args.des, ii)

    exp = Exp(args)  # 设置实验
    print('>>>>>>>测试 : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
    exp.test(setting, test=1)
    if args.use_gpu:
        torch.cuda.empty_cache()

Use CPU
>>>>>>>开始训练 : long_term_forecast_Stock_96_96_TimesNet_custom_ftMS_sl96_ll48_pl1_dm32_nh8_el2_dl1_df32_fc3_ebtimeF_dtTrue_test_0>>>>>>>>>>>>>>>>>>>>>>>>>>
train 36791
val 5270
test 10539


KeyboardInterrupt: 